In [ ]:
!pip install transformers tokenizers datasets

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
input_filename = "naslovi_new.input"
dataset_filename = input_filename + '.prepared'
pretrained_model_name = 'gpt2'
new_model_name = pretrained_model_name + '_2'

In [8]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(tokenizer_file="/content/drive/MyDrive/ML-Vesti/naslovi-simple-tokenizer.json")
tokenizer.pad_token = "<pad>"


In [9]:
from datasets import load_dataset
# datasets = load_dataset('text', data_files=dataset_filename, split="train[90%],validate")
eval, train = load_dataset('text', data_files='/content/drive/MyDrive/ML-Vesti/' + dataset_filename, split=['train[:10%]', 'train[10%:]'])
def prepare_for_trainer(example):
  tokenized = tokenizer(example['text'], truncation=True, padding='max_length', max_length=20)
  tokenized["labels"] = tokenized["input_ids"].copy()
  return tokenized
tokenized_train = train.map(prepare_for_trainer, num_proc=2, remove_columns=["text"])
tokenized_eval = eval.map(prepare_for_trainer, num_proc=2, remove_columns=["text"])


Using custom data configuration default-9b16dadd29989af1


Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-9b16dadd29989af1/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.
 

In [11]:
from transformers import Trainer, AutoConfig, AutoModelForCausalLM, TrainingArguments

config = AutoConfig.from_pretrained(pretrained_model_name)
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name)
# model.to('cuda:0')
training_args = TrainingArguments(
    # new_model_name,
    output_dir='/content/drive/MyDrive/ML-Vesti/' + new_model_name,
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    save_strategy='epoch',
    num_train_epochs=5,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    save_total_limit=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    # prediction_loss_only=True,
    tokenizer=tokenizer,
)


In [12]:
import logging
logger = logging.getLogger(__name__)
logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.local_rank,
        training_args.device,
        training_args.n_gpu,
        bool(training_args.local_rank != -1),
        training_args.fp16,
    )

Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False


In [ ]:
# import gc
# import torch 
# model = None
# trainer = None
# gc.collect()
# torch.cuda.empty_cache()

In [13]:
train_result = trainer.train()
trainer.save_model('/content/drive/MyDrive/ML-Vesti/modeli/' + new_model_name)

***** Running training *****
  Num examples = 1968703
  Num Epochs = 5
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 38455


Epoch,Training Loss,Validation Loss
1,3.417100,4.030136
2,3.218400,3.791500
3,3.108100,3.676917
4,3.063800,3.620540
5,3.035600,3.603560


***** Running Evaluation *****
  Num examples = 218745
  Batch size = 256
Saving model checkpoint to /content/drive/MyDrive/ML-Vesti/gpt2_2/checkpoint-7691
Configuration saved in /content/drive/MyDrive/ML-Vesti/gpt2_2/checkpoint-7691/config.json
Model weights saved in /content/drive/MyDrive/ML-Vesti/gpt2_2/checkpoint-7691/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/ML-Vesti/gpt2_2/checkpoint-7691/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/ML-Vesti/gpt2_2/checkpoint-7691/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 218745
  Batch size = 256
Saving model checkpoint to /content/drive/MyDrive/ML-Vesti/gpt2_2/checkpoint-15382
Configuration saved in /content/drive/MyDrive/ML-Vesti/gpt2_2/checkpoint-15382/config.json
Model weights saved in /content/drive/MyDrive/ML-Vesti/gpt2_2/checkpoint-15382/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/ML-Vesti/gpt2_2/checkpoint-15382/tokenizer_

In [14]:
import numpy as np
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/ML-Vesti/modeli/' + new_model_name)
model.config.pad_token_id = 1

for temp in np.linspace(0.2, 2, 10):
    print()
    print("###### Temperature: " + str(temp))
    for i in range(0, 5):
        out = model.generate(do_sample=True, temperature=temp)[0]
        print(tokenizer.decode(out, skip_special_tokens=True))

loading configuration file /content/drive/MyDrive/ML-Vesti/modeli/gpt2_2/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.8.2",
  "use_cache": true,
  "vocab_size": 50257
}

loading weights file /content/drive/MyDrive/ML-Vesti/mode


###### Temperature: 0.2
! biti!
biti! biti
biti
biti! biti
biti

###### Temperature: 0.4
u biti
,,,...
biti
i, i biti!
: je da, a da ne! biti

###### Temperature: 0.6000000000000001
se, a evo šta su joj poručili! ( VIDEO )
u! imati biti kao
,,
Partizan se, i biti kao u prvom meču
Sve je spremno za, ali su i! biti

###### Temperature: 0.8
Ovo je prvi put da znate koji srpski, u toku je se
Tramp čestitao Putinu rođendan biti na snazi način
za! biti na današnji dan u redu redu redu
! BITI!,
Evo kako da rešite da u, a da i za.

###### Temperature: 1.0
u borbi protiv koronavirusa se NA SUDU MESTU GODINE
se od muke bolesti! živeti i još jedan napad
U Nišu od sutra do doći što ga je čeka!
Muškarac uboden na Adi u Borči se videti ( VIDEO )
na smrt, u šoku ću umreti, a onda je na... ( VIDEO

###### Temperature: 1.2
Od sutra otvaranje bolnica za 100 evra morati da ga zbog jedne BOLESTI radnji
Evo zašto će sve izgledati padavine pacijenata na respiratoru večeri morati na virus,
Zvezda nastavlja 